In [1]:
!pip install beautifulsoup4

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from bs4 import BeautifulSoup
from bs4 import UnicodeDammit
import glob

In [0]:
pagesLocation = '/content/drive/My Drive/SITES BAIXADOS (Cris)/'

In [0]:
symbolsToReplace = [
    '(', ')', '!', '?', ':', ';', '*', '.', ',', '★', '|', '+', '[', ']', '{', '}', '/', 'ª', 'º', '°', '-',
    '%', '—', '@', '#',  '\"','\'', '<', '>', '=', '´', '`', '“', '$', '&', '’', '¡', '€', 'µ', '¦',
    '\\', '®', '™', '”', '…', '‘', '•', '😍','😙', '❤', '–', '⭐️', '_', '️⭐', '😉', '👏', '¹', '²', '³',
    "£", '¢', '¬', '§', '~', '^', '×', '÷',
    '\u200c', '\u200f', '\u200e', '\t', '\xa0', '\x03', '\u0301', '\u0303'
]

allowedSmallTokens = [
  'fio', 'box', 'god', 'key', 'bit', 'led', 'faq', 'dog', 'sim', 'ovo', 'pão', 'aba', 'new'
]

In [0]:
stopwords = []
with open(pagesLocation + 'stopwords.txt', 'r') as f:
    stopwords = [line.strip() for line in f]

In [0]:
def getAndCleanHTML(filepath):
  
  page = open(filepath, "rb")

  page = page.read()

  clearPage = UnicodeDammit.detwingle(page)
  
  doc = BeautifulSoup(clearPage, 'lxml')
  
  for script in doc(["script", "style"]):
    script.extract()
  
  docText = doc.get_text(' ')
  
  docText = docText.translate({ord(ch): None for ch in '0123456789'})
  
  docText = docText.lower()
  
  docText = docText.replace("e-book", "ebook")

  docText = docText.replace("blu-ray", "bluray")
  
  for symbol in symbolsToReplace:
    docText = docText.replace(symbol, ' ')
  
  docText = docText.replace("  ", ' ')
  
  lines = (line.strip() for line in docText.splitlines())
  
  chunks = (phrase.strip() for line in lines for phrase in line.split(' '))

  text = ' '.join(chunk for chunk in chunks if chunk)
  
  return text

In [0]:
allPositiveTexts = []
allNegativeTexts = []

In [0]:
positiveFilesList = [f for f in glob.glob(pagesLocation + "**/Positivas/*.html", recursive=True)]
negativeFilesList = [f for f in glob.glob(pagesLocation + "**/Negativas/*.html", recursive=True)]

In [17]:

  
def extractText(filesList, allTexts):
  for file in filesList:
    cleanText = getAndCleanHTML(file)
    allTexts.append(cleanText)

extractText(positiveFilesList, allPositiveTexts)
extractText(negativeFilesList, allNegativeTexts)

print(len(positiveFilesList), len(allPositiveTexts))

244 244


In [0]:
##import pprint

In [0]:
##%pprint

In [0]:
positiveVocab = dict()
negativeVocab = dict()

In [0]:
def textToTokenList(text):
  return text.split(' ')

def removeStopwords(list):
  return [word for word in list if word not in stopwords]

def removeSmallTokens(list):
  return [word for word in list if len(word)>3 or word in allowedSmallTokens ]

def updateVocabulary(token, vocab):
  if token in vocab:
    vocab.update({token: vocab.get(token) + 1})
  else:
    vocab[token] = 1

def buildUnorderedDict(allTexts, vocab):
  for text in allTexts:
    tokens = textToTokenList(text)
    tokens = removeStopwords(tokens)
    tokens = removeSmallTokens(tokens)
    for token in tokens:
      updateVocabulary(token, vocab)

def orderDictToTuplesList(unorderedDict):
  return sorted(unorderedDict.items(), key = lambda kv: kv[1], reverse=True)

In [0]:
buildUnorderedDict(allPositiveTexts, positiveVocab)
buildUnorderedDict(allNegativeTexts, negativeVocab)

In [0]:
##print(positiveVocab)

In [0]:
##print(negativeVocab)

In [0]:
common = positiveVocab.keys() & negativeVocab.keys()

def removeCommonTokens():
  for k in common:
    if positiveVocab.get(k) > negativeVocab.get(k):
      negativeVocab.pop(k, None)
    else:
      positiveVocab.pop(k, None)

removeCommonTokens()

In [0]:
def pluralToSingular(token):
  length = len(token)
  lastChar = token[length - 1]
  
  if lastChar != 's':
    return token
  
  last2Chars = token[length - 2] + lastChar
  last3Chars = token[length - 3] + last2Chars
  
  ## z   -> zes
  ## r   -> res
  ## m   -> ns ****
  ##*al  -> *ais
  ##*el  -> *eis ***
  ##*ol  -> *ois
  ##*ul  -> *uis
  ##*il  -> *is
  ##*s   -> *ses
  
  ##*ões <- ão
  ##*ães <- ão
  ##*ões <- ão
  
  singular = token
  
  if last3Chars == 'uis':
    #ul
    return token[0 : length - 2] + 'l'
  
  elif last3Chars == 'éis':
    #el
    return token[0 : length - 3] + 'el'
  
  elif last3Chars == 'óis':
    #ol
    return token[0 : length - 3] + 'ol'
    
  elif last3Chars == 'zes' or last3Chars == 'res' or last3Chars == 'ses':
    #z r s
    return token[0 : length - 2]
    
  elif last3Chars == 'ãos' or last3Chars == 'ães' or last3Chars == 'ões':
    #ão
    return token[0 : length - 3] + 'ão'
    
  elif last3Chars == 'ais' or last2Chars == 'ns' or last3Chars == 'eis' or last3Chars == 'ois' or last2Chars == 'ís':
    #ignorar
    return singular
  
  elif last2Chars == 'is':
    #il
    return token[0 : length - 1] + 'l'
  
  else:
    #tira o s
    return token[0 : length - 1]


In [0]:
def pluralOnlyStemming(vocab):
  d = vocab.copy()
  tokens = vocab.keys()
  
  for token in tokens:
    singular = pluralToSingular(token)
    if token != singular and singular in tokens:
      """print("Token: " + token)
      print("Singular: " + singular)"""
      d[singular] = d[singular] + d[token]
      ##print("Merged: " + singular + " + "  + token)
      d.pop(token)
      ##print("Popped: " + token)
      
  return d

In [0]:
stemmedPV = pluralOnlyStemming(positiveVocab)
stemmedNV = pluralOnlyStemming(negativeVocab)

In [0]:
##print(stemmedPV)

In [0]:
##print(stemmedNV)

In [0]:
def removeLeastOccurringTokens(vocab):
  newVocab = vocab.copy()
  keys = vocab.keys()
  
  for key in keys:
    if vocab[key] < 50:
      newVocab.pop(key)
  
  return newVocab

finalPV = removeLeastOccurringTokens(stemmedPV)
finalNV = removeLeastOccurringTokens(stemmedNV)

In [0]:
orderedPositiveVocab = orderDictToTuplesList(finalPV)


In [0]:
##print(orderedPositiveVocab)

In [0]:
orderedNegativeVocab = orderDictToTuplesList(finalNV)


In [0]:
##print(orderedNegativeVocab)

In [0]:
##print(common)

In [0]:
##len(orderedNegativeVocab)

# **CLASSIFICAÇÃO**

In [0]:
##allPagesPath = '/content/drive/My Drive/html misturado/'
##filesList = [f for f in glob.glob(pagesLocation + "*.html")]

In [39]:
!pip install joblib
!pip install numpy scipy matplotlib ipython jupyter pandas sympy nose
!pip install scikit-learn
!pip install scikit-plot

import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score

import scikitplot as skplt
import matplotlib
import matplotlib.pyplot as plt

     |████████████████████████████████| 163kB 2.8MB/s 


In [0]:
allTokens = set()
allTokens = allTokens.union(finalPV.keys())
allTokens = allTokens.union(finalNV.keys())

In [0]:
##print(allTokens)

In [0]:
#print(type(allTokens))

In [0]:
numDocs = len(positiveFilesList) + len(negativeFilesList)
numTokens = len(allTokens)

dados = np.zeros(shape=(numDocs, numTokens))
dataframe = pd.DataFrame(data=dados, columns=allTokens)
dataframe['CLASS'] = np.zeros(shape=(numDocs, 1))

In [0]:
##print(dataframe)

In [0]:
##df.columns
###Selecionando a primeira linha da coluna país: df.loc[0, 'País']

##print(len(allPositiveTexts))
everything = allPositiveTexts + allNegativeTexts
pLen = len(allPositiveTexts)
isBook = 1

def populateDataframe(isBook):
  for i,row in dataframe.iterrows():
    
    if i==pLen:
      isBook = 0
      ##print(isBook)
      
    dataframe.at[i, 'CLASS'] = isBook
    
    tokens = textToTokenList(everything[i])
    tokens = removeStopwords(tokens)
    tokens = removeSmallTokens(tokens)
    
    for token in tokens:
      s = pluralToSingular(token)
      
      if s in dataframe.columns:
        dataframe.at[i, s] += 1
        
      elif token in dataframe.columns:
        dataframe.at[i, token] += 1



populateDataframe(isBook)

In [0]:
##len(everything)

In [0]:
##len(allPositiveTexts)

In [0]:
##len(positiveFilesList)

In [0]:
##len(allNegativeTexts)

In [0]:
##len(negativeFilesList)

In [0]:
import time
timeNow = str(round(time.time() * 1000))

In [0]:
dataframe.to_csv(pagesLocation + 'PaginasClassificadas' + timeNow + '.csv', index = False)

In [0]:
##print(dataframe.columns)

In [0]:
##print(dataframe)

In [0]:
##print(dataframe.head())

In [0]:
csvPath = pagesLocation + 'PaginasClassificadas' + timeNow + '.csv'
csvData = np.genfromtxt(csvPath, delimiter=',', encoding='utf-8', dtype=None)

In [0]:
##print(csvData)

In [58]:
numDocs2 = len(positiveFilesList) + len(negativeFilesList)
numTokens2 = len(allTokens)

dados2 = np.zeros(shape=(numDocs2, numTokens2))
dataframe2 = pd.DataFrame(data=dados2, columns=allTokens)
dataframe2['CLASS'] = np.zeros(shape=(numDocs2, 1))


print(len(allPositiveTexts))
everything2 = allPositiveTexts + allNegativeTexts
pLen2 = len(allPositiveTexts)
isBook2 = 1

def populateDataframeAbs(isBook2):
  for i,row in dataframe2.iterrows():
    
    if i==pLen2:
      isBook2 = 0
      ##print(isBook)
      
    dataframe2.at[i, 'CLASS'] = isBook2
    
    tokens = textToTokenList(everything2[i])
    tokens = removeStopwords(tokens)
    tokens = removeSmallTokens(tokens)
    
    for token in tokens:
      s = pluralToSingular(token)
      
      if s in dataframe2.columns:
        dataframe2.at[i, s] = 1
        
      elif token in dataframe2.columns:
        dataframe2.at[i, token] = 1

populateDataframeAbs(isBook2)

timeNow2 = str(round(time.time() * 1000))
dataframe2.to_csv(pagesLocation + 'PaginasClassificadas_0e1_' + timeNow2 + '.csv', index = False)

244


In [0]:
##print(dataframe2)

In [63]:
from sklearn.naive_bayes import GaussianNB

csvPath2 = pagesLocation + 'PaginasClassificadas_0e1_' + timeNow2 + '.csv'
csvData2 = np.genfromtxt(csvPath2, delimiter=',', encoding='utf-8', dtype=None)

nb = GaussianNB()

##print("Shape: ", csvData2.shape)


Shape:  (489, 1386)


In [64]:
##print("Type: ", type(csvData2))

Type:  <class 'numpy.ndarray'>


In [0]:
dataSize = csvData.shape[1] - 1
dataColumns = csvData[1:, :dataSize - 1]
dataClasses = csvData[1:, dataSize]
dataColumnsF = dataColumns.astype(np.float)
dataClassesF = dataClasses.astype(np.float)

In [77]:
nb.fit(dataColumnsF, dataClassesF)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataColumnsF, dataClassesF, test_size=0.3, random_state=42)

In [121]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train, y_train)
classification = nb.predict(X_test)
score = nb.score(X_test, y_test)
print(score)

0.8163265306122449


In [167]:
accuracyNB1 = accuracy_score(y_test, classification)
print(accuracyNB1)

0.8163265306122449


In [122]:
from sklearn import tree
dt = tree.DecisionTreeClassifier()
dt.fit(X_train, y_train)
classfDT = dt.predict(X_test)
scoreDT = dt.score(X_test, y_test)

print(scoreDT)

0.9455782312925171


In [161]:
accuracyDT1 = accuracy_score(y_test, classfDT)
print(accuracyDT1)

0.9455782312925171


In [132]:
from sklearn.linear_model import LogisticRegressionCV
lr = LogisticRegressionCV(solver='sag', cv=5, max_iter=4000)
lr.fit(X_train, y_train)
classfLR = lr.predict(X_test)
scoreLR = lr.score(X_test, y_test)

print(scoreLR)

0.9387755102040817


In [163]:
accuracyLR1 = accuracy_score(y_test, classfLR)
print(accuracyLR1)

0.9387755102040817


0 e 1

In [0]:
dataSize2 = csvData2.shape[1] - 1
dataColumns2 = csvData2[1:, :dataSize2 - 1]
dataClasses2 = csvData2[1:, dataSize2]
dataColumnsF2 = dataColumns2.astype(np.float)
dataClassesF2 = dataClasses2.astype(np.float)

In [0]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(dataColumnsF2, dataClassesF2, test_size=0.3, random_state=42)

In [134]:
nb2 = GaussianNB()
nb2.fit(X_train2, y_train2)
classification2 = nb2.predict(X_test2)
score2 = nb2.score(X_test2, y_test2)
print(score2)

0.7891156462585034


In [164]:
accuracyNB2 = accuracy_score(y_test2, classification2)
print(accuracyNB2)

0.7891156462585034


In [135]:
dt2 = tree.DecisionTreeClassifier()
dt2.fit(X_train2, y_train2)
classfDT2 = dt2.predict(X_test2)
scoreDT2 = dt2.score(X_test2, y_test2)

print(scoreDT2)

0.9455782312925171


In [165]:
accuracyDT2 = accuracy_score(y_test2, classfDT2)
print(accuracyDT2)

0.9455782312925171


In [136]:
lr2 = LogisticRegressionCV(solver='sag', cv=5, max_iter=4000)
lr2.fit(X_train2, y_train2)
classfLR2 = lr2.predict(X_test2)
scoreLR2 = lr2.score(X_test2, y_test2)

print(scoreLR2)

0.9795918367346939


In [166]:
accuracyLR2 = accuracy_score(y_test2, classfLR2)
print(accuracyLR2)

0.9795918367346939
